# Data visualization of stocks

In [3]:
import numpy as np
import glob
import pandas as pd
from bokeh.io import output_notebook
from ipynb.fs.full.library_plots import plot_candlestick

output_notebook()

Loading BokehJS ...

## Find proper file based on symbol parameter

In [3]:
all_files = glob.glob(r"..\data\*.mst")
symbol = "XTB."
found_files = []
for path in all_files:
    if symbol in path:
        found_files.append(path)
found_files

['..\\data\\XTB.mst']

### Bulk plotter

In [4]:
observed = ["ONE"]
all_files = glob.glob(r"..\data\*.mst")
path_list = []
for item in observed:
    for path in all_files:
        if item in path and "RCF" not in path:
            path_list.append(path)
path_list

['..\\data\\OPONEO.PL.mst',
 '..\\data\\SONEL.mst',
 '..\\data\\EONET.mst',
 '..\\data\\MILESTONE.mst',
 '..\\data\\ONEMORE.mst',
 '..\\data\\GENXONE.mst']

In [1]:
import numpy as np
import glob
import pandas as pd
import nbimporter
from bokeh.io import output_notebook
from library_plots import plot_candlestick
output_notebook()

path_basked = ['..\\data\\CDPROJEKT.mst',
               '..\\data\\11BIT.mst',
               '..\\data\\XTB.mst',
               '..\\data\\BLOOBER.mst',
               '..\\data\\COLUMBUS.mst',
               '..\\data\\AIRWAY.mst',
               '..\\data\\UNIMOT.mst',
               '..\\data\\ONEMORE.mst',
               '..\\data\\DAX.mst']
date_start = "20200701"

# Plan: kupić więcej one more level lub unimot - zastanowić się nad blooberem?

for path in path_basked:
    data = pd.read_csv(path, index_col=1)
    data.index = pd.to_datetime(data.index, format="%Y%m%d")
    plot_candlestick(data, date_start, 11, 15, 10)

Importing Jupyter notebook from library_plots.ipynb


Loading BokehJS ...